## Evaluation Performance of Model

In [34]:
import pandas as pd
import numpy as np

def generate_waiting_list(capacity_df, referrals_df, stock_df, stage, num_weeks, df_TM):
    import pandas as pd
    """
    Generates the waiting list for a given stage.
    
    inputs: capacity_df: Capacity dataframe containing capacity for each stage
            referrals_df: Referrals dataframe containing the number of GP referrals for each priority
            stock_df: The stock (starting) waiting lists for each stage and each priority
            stage: The stage (location) that you want to generate the waiting list for
            num_weeks: Number of weeks you want to simulate the waiting list for
            
    outputs: df_WL: Contains the waiting list dataframe for the given stage. Also contains number
                    of patients seen for each priority and each simulated week
    """
    # initialise waiting list df
    d = {"week":list(range(0,num_weeks+1)),
         "twoWeek_WL":0,"urgent_WL":0,"routine_WL":0,
         "twoWeek_seen":0,"urgent_seen":0,"routine_seen":0}
    df_WL = pd.DataFrame(d)
    df_WL = df_WL.set_index("week")
    
    # set stock values as the waiting list value for week 0
    df_WL.loc[0,["twoWeek_WL", "urgent_WL", "routine_WL"]] = stock_df.loc[stage].values
    
    # get recurrent transition probability
    recurrent_prob = df_TM.loc[stage][stage]
    
    # for each week that we want to generate the waiting list
    for i in range(1, num_weeks+1):
        # get previous weeks waiting list value
        twoWeek_old_WL, urgent_old_WL, routine_old_WL = df_WL.loc[i-1,["twoWeek_WL", "urgent_WL", "routine_WL"]].values
        # get referral values
        twoWeek_referral, urgent_referral, routine_referral = referrals_df.loc[i].values
        # get capacity values
        capacity = capacity_df.loc[i,stage]
        
        # new twoWeek WL = (old twoWeek WL) + (twoWeek referrals) - (capacity used by twoWeek patients)
        # Capacity used is going to be the smaller of the avaiable capacity and old WL + referrals
        twoWeek_capacity_used = min(twoWeek_old_WL + twoWeek_referral, capacity)
        twoWeek_new_WL = twoWeek_old_WL + twoWeek_referral - twoWeek_capacity_used + twoWeek_capacity_used*recurrent_prob
        
        # new urgent WL = (old urgent WL) + (urgent referrals) - (capacity used by urgent patients)
        # capacity avaiable to urgent patients = total capacity - capacity used by 2week patients
        # therefore, capacity used by urgent patients = capacity available
        urgent_capacity_available = capacity - twoWeek_capacity_used
        urgent_capacity_used = min(urgent_old_WL + urgent_referral, urgent_capacity_available)
        urgent_new_WL = urgent_old_WL + urgent_referral - urgent_capacity_used + urgent_capacity_used*recurrent_prob
        
        # Same logic for routine
        routine_capacity_available = urgent_capacity_available - urgent_capacity_used
        routine_capacity_used = min(routine_old_WL + routine_referral, routine_capacity_available)
        routine_new_WL = routine_old_WL + routine_referral - routine_capacity_used + routine_capacity_used*recurrent_prob
        
        df_WL.loc[i] = [twoWeek_new_WL, urgent_new_WL, routine_new_WL,
                        twoWeek_capacity_used, urgent_capacity_used, routine_capacity_used]
          
    return df_WL

def inflow(from_stage, to_stage, from_df, df_TM_twoWeek, df_TM_urgent, df_TM_routine):
	import pandas as pd
	# Get transition probabilities for each priority  
	twoWeek_probability = df_TM_twoWeek.loc[from_stage,to_stage]
	urgent_probability = df_TM_urgent.loc[from_stage,to_stage]
	routine_probability = df_TM_routine.loc[from_stage,to_stage]
    
	# Get the number of patients seen at each priority
	if from_stage.startswith("referral"):
		twoWeek_seen = from_df.twoWeek_referrals.values
		urgent_seen = from_df.urgent_referrals.values
		routine_seen = from_df.routine_referrals.values
	else:
		twoWeek_seen = from_df.twoWeek_seen.values
		urgent_seen = from_df.urgent_seen.values
		routine_seen = from_df.routine_seen.values
    
	# Estimate inflow
	inflow = {"twoWeek_inflow":twoWeek_seen*twoWeek_probability,
              "urgent_inflow":urgent_seen*urgent_probability,
              "routine_inflow":routine_seen*routine_probability}
    
	if from_stage.startswith("referral"):
		inflow_df = pd.DataFrame(inflow)
		inflow_df.index += 1
	else:
		inflow_df = pd.DataFrame(inflow).loc[1:]
    
	return inflow_df

In [35]:
####### Retrieve capacity parameters

cap_dir = "//wwlqlikview1/_Everyone/SharedDataFiles/WL_modelling/Waiting_List_Capacity.xlsx"
df_capacity = pd.read_excel(
    cap_dir,
    index_col="week")
weeks = len(df_capacity)

In [36]:
####### Make dataframe of outpatient and inpatient referrals

d_dict = {
    "week": list(range(1, weeks + 1)),
    "twoWeek_referrals": np.repeat(29,
                                   weeks),
    "urgent_referrals": np.repeat(31, weeks),
    "routine_referrals": np.repeat(73,
                                   weeks)
}
df_referrals_op = pd.DataFrame(d_dict).set_index("week")

d_dict = {
    "week": list(range(1, weeks + 1)),
    "twoWeek_referrals": np.repeat(0,
                                   weeks),
    "urgent_referrals": np.repeat(94, weeks),
    "routine_referrals": np.repeat(116,
                                   weeks)
}
df_referrals_ip = pd.DataFrame(d_dict).set_index("week")

In [37]:
####### Make dataframe of inpatient and outpatient current waiting list (stock)

d_dict = {
    "stage": df_capacity.columns.values,
    "twoWeek":[0,412,82],
    "urgent":[305,940,483],
    "routine":[2615,4062,2864]
}
df_stock = pd.DataFrame(d_dict).set_index("stage")

In [38]:
####### Make transition matrices (one for each priority) for probability of walking between stages

d_dict = {
    "idx": ["referral", "referral_ip", "inpatient", "Discharge", "First", "Follow Up"],
    "referral": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    "referral_ip": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    "inpatient": [0, 1, 0, 0.0, 0, 0],
    "Discharge": [0.58, 0.0, 1, 0.0, 0, 0],
    "First": [0.38, 0.0, 0, 0.0, 0.01, 0.01],
    "Follow Up": [0.04, 0.0, 0.0, 0.0, 0.2, 0.2]
}
df_TM_twoWeek = pd.DataFrame(d_dict).set_index("idx")
df_TM_urgent = df_TM_twoWeek.copy()
df_TM_routine = df_TM_twoWeek.copy()

In [39]:
####### Generate waiting lists

# Referrals to First
inflow_referrals_First = inflow("referral", "First", df_referrals_op, df_TM_twoWeek, df_TM_urgent, df_TM_routine)
df_WL_First = generate_waiting_list(df_capacity, inflow_referrals_First, df_stock, "First", weeks, df_TM_routine)

# First to Follow Up
inflow_First_FU = inflow("First", "Follow Up", df_WL_First, df_TM_twoWeek, df_TM_urgent, df_TM_routine)
df_WL_FU = generate_waiting_list(df_capacity, inflow_First_FU, df_stock, "Follow Up", weeks, df_TM_routine)

# Inpatient Referrals to Inpatient Spell
inflow_ip_referrals_inp = inflow("referral_ip", "inpatient", df_referrals_ip, df_TM_twoWeek, df_TM_urgent, df_TM_routine)
df_WL_inp = generate_waiting_list(df_capacity, inflow_ip_referrals_inp, df_stock, "inpatient", weeks, df_TM_routine)

In [40]:
# Put data into single dataframe to be returned to qlik

df_WL_First = df_WL_First.add_suffix("_First")
df_WL_inp = df_WL_inp.add_suffix("_inp")
df_WL_FU = df_WL_FU.add_suffix("_FU")
df_capacity = df_capacity.add_suffix("_capacity")
df_referrals_ip = df_referrals_ip.add_suffix("_ip")
df_referrals_op = df_referrals_op.add_suffix("_op")

dfs = [df_WL_First, df_WL_inp, df_WL_FU, df_referrals_ip, df_referrals_op, df_capacity]
return_df = pd.concat(dfs, join="inner", axis=1)
return_df.reset_index(level=0, inplace=True)

#start_week = pd.Timestamp.today().date() - pd.DateOffset(days=pd.Timestamp.today().dayofweek)
start_week = pd.Timestamp(month=10,year=2021,day=25)
date_refs = []
for i in range(weeks):
	date_refs.append((start_week + pd.DateOffset(days=7*i)).strftime("%d/%m/%Y"))
return_df["snapshot_date_dt"] = date_refs

In [41]:
return_df

,week,twoWeek_WL_First,urgent_WL_First,routine_WL_First,twoWeek_seen_First,urgent_seen_First,routine_seen_First,twoWeek_WL_inp,urgent_WL_inp,routine_WL_inp,...,twoWeek_referrals_ip,urgent_referrals_ip,routine_referrals_ip,twoWeek_referrals_op,urgent_referrals_op,routine_referrals_op,inpatient_capacity,First_capacity,Follow Up_capacity,snapshot_date_dt
0,1,348.770000,951.780000,4089.74,75.000000,0.000000,0,0,242,2731,...,0,94,116,29,31,73,157,75,115,25/10/2021
1,2,286.530000,963.560000,4117.48,74.000000,0.000000,0,0,180,2847,...,0,94,116,29,31,73,156,74,118,01/11/2021
2,3,224.290000,975.340000,4145.22,74.000000,0.000000,0,0,120,2963,...,0,94,116,29,31,73,154,74,118,08/11/2021
3,4,162.050000,987.120000,4172.96,74.000000,0.000000,0,0,76,3079,...,0,94,116,29,31,73,138,74,118,15/11/2021
4,5,99.810000,998.900000,4200.70,74.000000,0.000000,0,0,41,3195,...,0,94,116,29,31,73,129,74,118,22/11/2021
5,6,37.570000,1010.680000,4228.44,74.000000,0.000000,0,0,0,3290,...,0,94,116,29,31,73,156,74,118,29/11/2021
6,7,0.485900,997.304100,4256.18,48.590000,25.410000,0,0,0,3344,...,0,94,116,29,31,73,156,74,118,06/12/2021
7,8,0.115059,947.214941,4283.92,11.505900,62.494100,0,0,0,3422,...,0,94,116,29,31,73,132,74,118,13/12/2021
8,9,0.111351,896.758649,4311.66,11.135059,62.864941,0,0,0,3476,...,0,94,116,29,31,73,156,74,118,20/12/2021
9,10,0.111314,846.298686,4339.40,11.131351,62.868649,0,0,0,3530,...,0,94,116,29,31,73,156,74,118,27/12/2021


In [67]:
df_WL_First.iloc[:,0:3].sum(axis=1)+df_WL_inp.iloc[:,0:3].sum(axis=1)+df_WL_FU.iloc[:,0:3].sum(axis=1)

week
0     11763.00
1     11715.29
2     11666.97
3     11620.65
4     11590.33
5     11569.01
6     11520.69
7     11472.37
8     11448.05
9     11399.73
10    11351.41
11    11303.09
12    11254.77
dtype: float64